Notebook configuration

Import the required libraries here


In [1]:
import pandas as pd
import requests
import json

from datetime import datetime
from dateutil.relativedelta import relativedelta
from time import sleepures


 API

In [2]:
endpoint_url = 'https://##########################.pl/api/v2'
token = '#################################'
request_offset = 0.3

authorization = {'authorization': token}

In [3]:
airport_list = pd.read_csv('../data/airports.csv')
airports = airport_list['origin_airport_id'].values.tolist()

In [ ]:
airport_endpoint_url = f"{endpoint_url}/airport"

In [4]:
airport_data = []
for airport_id in airports:
    sleep(request_offset)

    r = requests.get(
        f"{airport_endpoint_url}/{airport_id}",
        headers=authorization,
    )

    if r.status_code != 200:
        continue

    response = json.loads(r.text)
    airport_data.append(response)

Error: Could not retrieve data for airport with ID 10874
Error: Could not retrieve data for airport with ID 11233
Error: Could not retrieve data for airport with ID 13360
Error: Could not retrieve data for airport with ID 15008
Error: Could not retrieve data for airport with ID 14150
Error: Could not retrieve data for airport with ID 15323
Error: Could not retrieve data for airport with ID 14814
Error: Could not retrieve data for airport with ID 12007
Error: Could not retrieve data for airport with ID 11337
Error: Could not retrieve data for airport with ID 15070
Error: Could not retrieve data for airport with ID 12280
Error: Could not retrieve data for airport with ID 11641
Error: Could not retrieve data for airport with ID 13832
Error: Could not retrieve data for airport with ID 10268
Error: Could not retrieve data for airport with ID 15041
Error: Could not retrieve data for airport with ID 12119
Error: Could not retrieve data for airport with ID 11537
Error: Could not retrieve data 

Error: Could not retrieve data for airport with ID 15024
Error: Could not retrieve data for airport with ID 12917
Error: Could not retrieve data for airport with ID 12124
Error: Could not retrieve data for airport with ID 12523
Error: Could not retrieve data for airport with ID 11630
Error: Could not retrieve data for airport with ID 10170
Error: Could not retrieve data for airport with ID 12448
Error: Could not retrieve data for airport with ID 10918
Error: Could not retrieve data for airport with ID 15356
Error: Could not retrieve data for airport with ID 13344
Error: Could not retrieve data for airport with ID 10676
Error: Could not retrieve data for airport with ID 10926
Error: Could not retrieve data for airport with ID 12156
Error: Could not retrieve data for airport with ID 12244
Error: Could not retrieve data for airport with ID 10666
Error: Could not retrieve data for airport with ID 11612
Error: Could not retrieve data for airport with ID 13476
Error: Could not retrieve data 

In [ ]:
airport_df = pd.DataFrame.from_records(airport_data)
airport_df.head()

Check:

In [6]:
airport_df_expected_shape = (97, 4)
assert airport_df_expected_shape == airport_df.shape

In [7]:
airport_df.to_csv("..\\data\\raw\\airport_list.csv", index=False)

 # Download `Weather`


In [8]:
months_amount = 15
start_date = datetime(2019, 1, 1)
airport_weather_endpoint_url = f"{endpoint_url}/airportWeather"

Successfully fetched data for 2019-01
Successfully fetched data for 2019-02
Successfully fetched data for 2019-03
Successfully fetched data for 2019-04
Successfully fetched data for 2019-05
Successfully fetched data for 2019-06
Successfully fetched data for 2019-07
Successfully fetched data for 2019-08
Successfully fetched data for 2019-09
Successfully fetched data for 2019-10
Successfully fetched data for 2019-11
Successfully fetched data for 2019-12
Successfully fetched data for 2020-01
Successfully fetched data for 2020-02
Successfully fetched data for 2020-03


In [ ]:
def get_weather_data(date):
    r = requests.get(
        airport_weather_endpoint_url,
        headers=authorization,
        params={'date': date.strftime('%Y-%m')}
    )

    print(date.strftime('%Y-%m'))
    response = json.loads(r.text)

    return response

In [ ]:
airport_weather_data = []

for month in range(months_amount):
    print(f"Pobieram {month+1} z {months_amount}")
    date = start_date + relativedelta(months=month)
    daily_weather = get_weather_data(date)
    print(len(daily_weather))

    airport_weather_data += daily_weather
    sleep(request_offset)

In [ ]:
airport_weather_df = pd.DataFrame.from_records(airport_weather_data)
airport_weather_df.head()

 ## Check


In [9]:
airport_weather_df_expected_shape = (46226, 33)
assert airport_weather_df_expected_shape == airport_weather_df.shape

 ## save to file

In [10]:
airport_weather_df.to_csv('..\\data\\raw\\airport_weather.csv', index=False)

 # Download `Aircraft`
 


In [11]:
aircraft_endpoint_url = f"{endpoint_url}/aircraft"
r = requests.get(aircraft_endpoint_url, headers=authorization)
data = json.loads(r.text)

aircraft_df = pd.DataFrame.from_records(data)
aircraft_df.head()

   MANUFACTURE_YEAR TAIL_NUM  NUMBER_OF_SEATS
0              1944   N54514              0.0
1              1945   N1651M              0.0
2              1953   N100CE              0.0
3              1953   N141FL              0.0
4              1953   N151FL              0.0


 ## Check


In [12]:
aircraft_df_expected_shape = (7383, 3)
assert aircraft_df_expected_shape == aircraft_df.shape

 ## Save to file

In [13]:
aircraft_df.to_csv('..\\data\\raw\\aircraft.csv', index=False)


 # Download `Flight`
 

In [ ]:
endpoint_url = 'https://api-#############/api'
token = 'WpzDMZeeCq6tbPdsTHUX8W9mecuUVwXAnmcorefr'
request_offset = 0.3
authorization = {'authorization': token}

flight_endpoint_url = f"{endpoint_url}/flight"

def get_airport_data(airport_id):
    result = []
    start_date = datetime(2019, 1, 1)
    end_date = datetime.now()
    months_amount = (end_date.year - start_date.year) * 12 + end_date.month - start_date.month + 1

    for month in range(months_amount):
        date = start_date + relativedelta(months=month)
        print(f"\t{date}")

        response = get_airport_day_data(airport_id, date)
        result += response

        sleep(request_offset)

    return result

def get_airport_day_data(airport_id, date):
    payload = {
        'airportId': airport_id,
        'date': date.strftime('%Y-%m')
    }

    r = requests.get(
        flight_endpoint_url,
        headers=authorization,
        params=payload)

    response = json.loads(r.text)
    return response

airports = pd.read_csv(r"..\\data\\airports.csv")


flight = []
airports_amount = len(airports)

for idx, airport in enumerate(airports.to_dict(orient='records')):
    print(f"{airport}: {idx + 1} of {airports_amount}")
    airport_flight = get_airport_data(airport["origin_airport_id"])
    flight += airport_flight
    
flight_df = pd.DataFrame.from_records(flight)
flight_df.head()

 ## Check


In [ ]:
flight_df_expected_shape = (9251880, 27)
assert flight_df_expected_shape == flight_df.shape

 ## Save to file
  `data/raw`.

In [ ]:
flight_df.to_csv('flight.csv')

 # Summary

In [ ]:
msg = "Everything looks good :))"
print(msg)